In [1]:
import sys
configDir =  '/home/thibaud/repos/anaconda/rakuten/notebook'
if configDir not in sys.path:
    sys.path.append(configDir)

import config


import warnings
warnings.filterwarnings('ignore')
from importlib import reload
import src.utils.results as results
from sklearn.metrics import f1_score

# print(config.path_to_data)
reload(results)

from src.utils.results import ResultsManager
res = ResultsManager(config)
res.add_result_file(config.path_to_results +
                        '/results_benchmark_cbow.csv', 'text')
res.add_result_file(config.path_to_results +
                    '/results_benchmark_tfidf.csv', 'text')
res.add_result_file(config.path_to_results +
                    '/results_benchmark_skipgram.csv', 'text')

res.add_result_file(config.path_to_results +
                    '/results_benchmark_bert.csv', 'bert')
res.add_result_file(config.path_to_results +
                    '/results_benchmark_img.csv', 'img')
res.add_result_file(
    config.path_to_results+'/results_benchmark_fusion_TF.csv', 'fusion')

2024-03-17 18:27:30.133446: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-17 18:27:30.157313: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-17 18:27:31.409493: I itex/core/wrapper/itex_gpu_wrapper.cc:35] Intel Extension for Tensorflow* GPU backend is loaded.
2024-03-17 18:27:31.438332: I itex/core/wrapper/itex_cpu_wrapper.cc:52] Intel Extension for Tensorflow* AVX2 CPU backend is loaded.
2024-03-17 18:27:31.482371: W itex/core/ops/op_init.cc:58] Op: _QuantizedMaxPool3D is already registere

In [2]:
import utils.scrapper as scrapper

scrap = scrapper.RakutenScrapper()

des, desc, img = scrap.get_rakuten_product_infos('https://fr.shopping.rakuten.com/offer/shop/5640308610/nendoroid-doll-harry-potter-ron-weasley-import-japonais.html?sellerLogin=GAMERZ')

print(des)
print(desc)
print(img)

Good Smile Company HARRY POTTER Nendoroid
Figurine Nendoroid de Ron Weasley.Taille approx: 10cm.
https://fr.shopping.rakuten.com/photo/nendoroid-doll-harry-potter-ron-weasley-import-japonais-1693345551_L.jpg


In [2]:
res.get_model_paths()

array(['text/LogisticRegression_cbow', 'text/RandomForestClassifier_cbow',
       'text/SVC_cbow', 'text/xgboost_cbow', 'text/SVC_skipgram',
       'text/LogisticRegression_skipgram',
       'text/RandomForestClassifier_skipgram', 'text/xgboost_skipgram',
       'text/LinearSVC_tfidf', 'text/LogisticRegression_tfidf',
       'text/MultinomialNB_tfidf', 'text/RandomForestClassifier_tfidf',
       'text/xgboost_tfidf', 'text/SVC_tfidf',
       'text/camembert-base-ccnet', 'text/camembert-base',
       'text/flaubert_base_uncased', 'image/vit_b16', 'image/ResNet152',
       'image/ResNet101', 'image/EfficientNetB1', 'image/ResNet50',
       'image/VGG16', 'fusion/camembert-base-vit_b16_TF6',
       'fusion/camembert-base-vit_b16_TF3_att12',
       'fusion/camembert-base-vit_b16_TF6_att12',
       'fusion/camembert-base-vit_b16_TF1_att12'], dtype=object)

In [10]:
# res.plot_classification_report('text/RandomForestClassifier_skipgram')
    # 

In [13]:
from bs4 import BeautifulSoup
def scrap_rakuten_page(page_content):
    # your code here
    soup = BeautifulSoup(page_content, 'html.parser')
    if len(soup.select('#prdTitleHead > span.detailHeadline')) > 0:
        designation = soup.select(
        '#prdTitleHead > span.detailHeadline')[0].get_text().strip()
    else:
        designation = 'not found'
    if len(soup.select('div.fullDescription span.edito')) > 0:
        description = soup.select(
            "div.fullDescription span.edito")[0].get_text().strip()
    else:
        description = 'not found'
    if len(soup.select(".prdMainPhoto")) > 0:
        image_path = soup.select(".prdMainPhoto")[0].get('href')
    else:
        image_path = 'not found'
    return designation, description, image_path

des,desc,img = scrap_rakuten_page(html)
print(des,desc,img)

Good Smile Company HARRY POTTER Nendoroid Figurine Nendoroid de Ron Weasley.Taille approx: 10cm. https://fr.shopping.rakuten.com/photo/nendoroid-doll-harry-potter-ron-weasley-import-japonais-1693345551_L.jpg


In [11]:
from PIL import Image
import pandas as pd

path_to_demo_img = [config.path_to_data + '/demo/demo.avif', config.path_to_data + '/demo/demo2.avif', config.path_to_data + '/demo/demo3.avif']
designations = [
    "Piscine tubulaire 396 x 84 cm Cette piscine tubulaire ronde mesure ø3,96Mx0,84M avec une finition imitation tressée sur son liner fabriqué en PVC triple épaisseur et sa structure en acier qui la rende robuste. Elle possède deux hublots pour voir en dehors à ses extrémités. Inclus pompe",
    "Console Sony Playstation 5 Slim Edition Digitale",
    "Good Smile Company HARRY POTTER Nendoroid "
    ]
# tokens = ' '.join(s.lower() for s in designation)

data = pd.DataFrame({'tokens': designations, 'img_path': path_to_demo_img})
data.head()
pred = res.predict('text/LinearSVC_tfidf', data)
# pred = res.get_y_pred('text/LinearSVC_tfidf')
display(pred)



array(['Piscines et accessoires', 'Consoles de jeu',
       'Figurines et objet pop culture'], dtype='<U30')

In [4]:
import sys
import os
from requests_html import HTMLSession
configDir =  '/home/thibaud/repos/anaconda/rakuten/notebook'
if configDir not in sys.path:
    sys.path.append(configDir)

import config
from bs4 import BeautifulSoup as bs

response = s
# res = driver.get('https://fr.shopping.rakuten.com/mfp/shop/9799998/console-sony-playstation-5-slim?pid=11945408653&sellerLogin=Carrefour&fbbaid=15008141157&rd=1')
# print(res.content)
# soup = bs(res.content, 'lxml')
# soup.find('div', class_='prdGallery').find_all('img')

# print(soup)

## .prdGallery 


WebDriverException: Message: Process unexpectedly closed with status 1


In [6]:
# !pip install requests-html


In [7]:
import asyncio
from requests_html import AsyncHTMLSession
from bs4 import BeautifulSoup
import nest_asyncio
nest_asyncio.apply()

async def main():
    # Create an async HTML session
    session = AsyncHTMLSession()

    # Fetch the webpage and render JavaScript
    response = await session.get('https://example.com', headers={'User-Agent': 'Mozilla/5.0'})

    # Ensure JavaScript is fully rendered
    await response.html.arender()

    # Now you can parse the HTML with BeautifulSoup
    soup = BeautifulSoup(response.html.html, 'html.parser')

    # Now you can use BeautifulSoup to extract information from the rendered HTML
    
# Now you can run asyncio code in the Jupyter Notebook
await main()


/home/thibaud/intel/oneapi/intelpython/envs/tensorflow/lib/python3.9/site-packages/pyppeteer/util.py:29: RuntimeWarning: coroutine 'main' was never awaited
  gc.collect()
[INFO] Starting Chromium download.
100%|██████████| 183M/183M [01:18<00:00, 2.33Mb/s] 
[INFO] Beginning extraction
[INFO] Chromium extracted to: /home/thibaud/.local/share/pyppeteer/local-chromium/1181205


In [24]:
import sys
import os
configDir =  '/home/thibaud/repos/anaconda/rakuten/notebook'
if configDir not in sys.path:
    sys.path.append(configDir)

import config

from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager

# path_to_firefox = config.path_to_project + '/bin/geckodriver --profile "' + config.path_to_project + '/bin/firefox_profile"'
path_to_firefox = config.path_to_project + '/bin/geckodriver'
# print(path_to_firefox)
service = Service(executable_path=path_to_firefox)

options = webdriver.FirefoxOptions()
# profile = webdriver.FirefoxProfile()
options.set_preference("profile", config.path_to_project + '/bin/firefox_profile')

# options.add_argument("-profile")
# options.add_argument(config.path_to_project + '/firefox_profile')


# Initialize WebDriver with GeckoDriverManager to automatically download and manage GeckoDriver
driver = webdriver.Firefox(service=service, options=options)

# Navigate to a website
driver.get("http://www.python.org")

# Verify that the page title contains "Python"
assert "Python" in driver.title

# Close the WebDriver session
driver.quit()

WebDriverException: Message: Process unexpectedly closed with status 1


In [25]:
import sys
import os
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager

# Add the directory containing the config file to sys.path
configDir = '/home/thibaud/repos/anaconda/rakuten/notebook'
if configDir not in sys.path:
    sys.path.append(configDir)

# Import config module from the directory
import config

# Set the path to GeckoDriver
path_to_geckodriver = config.path_to_project + '/bin/geckodriver'

# Initialize the WebDriver service
service = Service(executable_path=path_to_geckodriver)

# Set Firefox profile preferences
firefox_options = webdriver.FirefoxOptions()
firefox_options.set_preference("profile", config.path_to_project + '/bin/firefox_profile')
firefox_profile = webdriver.FirefoxProfile()
# Initialize WebDriver with GeckoDriverManager to automatically download and manage GeckoDriver
driver = webdriver.Firefox(service=service, options=firefox_options)

# Navigate to a website
driver.get("http://www.python.org")

# Verify that the page title contains "Python"
assert "Python" in driver.title

# Close the WebDriver session
driver.quit()


WebDriverException: Message: Process unexpectedly closed with status 1


In [26]:
from selenium import webdriver

# Create a new Firefox profile
firefox_profile = webdriver.FirefoxProfile()

# Set preferences for the Firefox profile (if needed)
# firefox_profile.set_preference("key", "value")

# Initialize WebDriver with the new profile
driver = webdriver.Firefox(firefox_profile=firefox_profile)

# Navigate to a website
driver.get("http://www.python.org")

# Verify that the page title contains "Python"
assert "Python" in driver.title

# Close the WebDriver session
driver.quit()


TypeError: __init__() got an unexpected keyword argument 'firefox_profile'

In [6]:
!pip install selenium==4.18.1 webdriver_manager


In [2]:
import pandas as pd
pd.set_option('display.max_colwidth',2000)
df = pd.read_csv('../../data/clean/df_train_index.csv')
df.head()

,Unnamed: 0,designation,description,productid,imageid,language,designation_translated,description_translated,prdtypecode,prdtypedesignation,prdtypefull,prdtypeindex
0,0,Porte Bébé Violet Et Rouge Trois-En-Un Mère Multifonctions Kangourou Fermeture À Glissière Hoodie Taille: Xl Poitrine: 104-109 Cm 84-88 Cm Hanche: 110-116 Cm Clair +,Porte bébé Violet et rouge Trois-en-un mère multifonctions Kangourou fermeture à glissière Hoodie Taille: XL Poitrine: 104-109 cm 84-88 cm Hanche: 110-116 cm clair + 1. Marque nouvelle et de haute qualité. 2. Détachable conception pratique et attentionnée. 3. Parfait pour les mères qui allaitent. 4. Anti-vent chaud et style kangourou multifonctionnel haut de gamme. 5. Sac de couchage multifonction amovible de la mère européenne. Spécification: Les types Fermez Buste104-109cm Encolure Sweat à capuche Les hanches110-116cm Tailles disponiblesXLMatériel Coton,3050424970,1187504001,fr,Porte Bébé Violet Et Rouge Trois-En-Un Mère Multifonctions Kangourou Fermeture À Glissière Hoodie Taille: Xl Poitrine: 104-109 Cm 84-88 Cm Hanche: 110-116 Cm Clair +,Porte bébé Violet et rouge Trois-en-un mère multifonctions Kangourou fermeture à glissière Hoodie Taille: XL Poitrine: 104-109 cm 84-88 cm Hanche: 110-116 cm clair + 1. Marque nouvelle et de haute qualité. 2. Détachable conception pratique et attentionnée. 3. Parfait pour les mères qui allaitent. 4. Anti-vent chaud et style kangourou multifonctionnel haut de gamme. 5. Sac de couchage multifonction amovible de la mère européenne. Spécification: Les types Fermez Buste104-109cm Encolure Sweat à capuche Les hanches110-116cm Tailles disponiblesXLMatériel Coton,1320,Puériculture,1320 - Puériculture,9
1,1,Jesus - Cahiers Du Libre Avenir,Prêtre autrement.,131641431,885888766,fr,Jesus - Cahiers Du Libre Avenir,Prêtre autrement.,10,Livres occasion,10 - Livres occasion,0
2,2,Chambre Paillasson En Forme De Coeur Tapis Fluffy Tapis Chenille Coussin Jaune Kitchen Supplies 291,Chambre Paillasson en forme de coeur Tapis Tapis Coussin Fluffy Chenille Feature: 100% qualité de la marque nouvelle et haute. Quantité: 1 Designer et matériaux de haute qualité Matériaux: Corail Taille polaire: 40 * 50cm Contenu: 1 x Bedroom Paillasson en forme de coeur Tapis Fluffy Tapis Chenille Coussin,4197486437,1313030973,fr,Chambre Paillasson En Forme De Coeur Tapis Fluffy Tapis Chenille Coussin Jaune Kitchen Supplies 291,Chambre Paillasson en forme de coeur Tapis Tapis Coussin Fluffy Chenille Feature: 100% qualité de la marque nouvelle et haute. Quantité: 1 Designer et matériaux de haute qualité Matériaux: Corail Taille polaire: 40 * 50cm Contenu: 1 x Bedroom Paillasson en forme de coeur Tapis Fluffy Tapis Chenille Coussin,1560,Mobilier,1560 - Mobilier,10
3,3,2pcs En Alliage D'aluminium Portail Du Carter D'entraînement Avant Intérieur Pour Traxxas Trx-4 1/10 Rc,2pcs en alliage d'aluminium Portail du carter d'entraînement intérieur avant TRAXXAS TRX4 1/10 RC Feature: la marque 100% nouvelle de haute qualité Ce hub de direction en alliage d'aluminium de haute qualité mise à niveau de montage set spécialement conçu pour TRX4 1/10 voiture RC. apparence originale et l'exécution exquise remplacer directement TRX4 8252 avant plactis intérieure Portail du carter d'entraînement avec les paliers il protégera les arbres et faire tourner la direction plus efficace. Facile à installer et durable dans l'utilisation légère mise à jour améliore instantanément la performance globale dans la manipulation et la durabilité Fabriqué en alliage d'aluminium conçu pour réparer ou remplacer centre ancien ou cassé montage comprennent Emballage: 2 x Hub directeur,3929174950,1265009801,fr,2pcs En Alliage D'aluminium Portail Du Carter D'entraînement Avant Intérieur Pour Traxxas Trx-4 1/10 Rc,2pcs en alliage d'aluminium Portail du carter d'entraînement intérieur avant TRAXXAS TRX4 1/10 RC Feature: la marque 100% nouvelle de haute qualité Ce hub de direction en alliage d'aluminium de haute qualité mise à niveau de montage se